In [2]:
import matplotlib.pyplot as plt
import torch
import torch.distributions as torch_d
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import numpy as np

from pcs import tx
from pcs import helper as hlp
from pcs import utils
from pcs import autoencoder as ae

# Parameters

In [3]:
# Channel Parameters
chParam = utils.AttrDict()
chParam.M = 64
# chParam.SNR_db = [5, 12 ]
chParam.SNR_db = [0.5, 1, 3, 5, 7, 12, 30]

# Auto-Encoder Parameters
aeParam = utils.AttrDict()
aeParam.nLayersEnc  = 1
aeParam.nLayersDec  = 5
aeParam.nFeaturesEnc  = 128
aeParam.nFeaturesDec  = 128

# Training Parameters
trainingParam = utils.AttrDict()
trainingParam.nBatches      = 16
trainingParam.batchSize     = 128*chParam.M
trainingParam.learningRate  = 0.0001
trainingParam.iterations    = 31
trainingParam.displayStep   = 5

NameError: name 'utils' is not defined

## Helper Functions

In [23]:
def p_norm(p, x, fun=lambda x: torch.pow(torch.abs(x), 2)):
    return torch.sum(p * fun(x))

In [24]:
def sampler(P_M, B):
    samples = np.zeros((np.sum(np.rint(B*P_M).astype(int)), 0))
    total_occurrences = np.sum(np.rint(B * P_M).astype(int))
    for idx, p in enumerate(P_M):
        occurrences = np.rint(B*p).astype(int)
        samples = np.append(samples, np.ones(occurrences)*(idx))
    np.random.shuffle(samples)
    return samples

In [25]:
def sampler_t(P_M, B):
    samples = torch.empty(0)
    for idx, p in enumerate(P_M):
        occurrences = torch.round(B*p).type(torch.LongTensor)
        samples = torch.cat((samples, torch.ones(occurrences, dtype=torch.int64)*torch.tensor(idx)))
    indexes = torch.randperm(samples.shape[0])
    return samples[indexes]

In [26]:
def calculate_py_given_x(z, sig2):
    return (1 / (torch.sqrt(2 * torch.pi * sig2))) * torch.exp(-torch.square(z) / (sig2 * torch.tensor(2)))

## Loss Function
The Cross Equivocation must be corrected with an additional term to account for the changing statistics of the information source:
$$CE_{loss} + \sum \limits_{Y_{n}} P(Y_{n}|c_{i})log_{2}(Q_{\theta,P_{M},C_{M}}(c_{m}|Y_{n})$$
Regarding the prior term, the complex representation is reduced from (N,2) to N by means of multiplication. This is because we have a complex AWGN where the Noise is circularly symmetric.
This means:
$$P(Y_{R}Y_{I}|X_{R}X_{I}) = P(Y_{R}|X_{R})P(Y_{I}|_{R}X_{I})$$
While the posterior term is computed from the learned distribution and taking the max.

In [27]:
# CE loss function and correct with additional term
def loss_correction_factor(dec, zhat, sig2):
    q = torch.amax(dec, 1)  # Q(c_i|Y_n) <-- learned
    p = torch.prod(calculate_py_given_x(zhat, sig2), 1)  # P(Y_n|c_i)
    return torch.mean(p * torch.log2(q))

In [28]:
def r2c(x):
    #a = torch.tensor(x, dtype=torch.double)
    return x.type(torch.complex64)

In [29]:
def plot_2D_PDF(const, pmf, db):
    s = pmf * 400
    plt.figure(figsize=(5, 5))
    plt.scatter(const.real, const.imag, s, c="r")
    plt.title(f'SNR = {db} dB')
    plt.grid()
    plt.show()

## Training Loop

In [30]:
enc_inp = torch.tensor([[1]], dtype=torch.float)

for (k, SNR_db) in enumerate(chParam.SNR_db):
    print(f'---SNR = {chParam.SNR_db[k]} dB---')

    # Initialize network
    encoder = ae.Encoder_Aref(in_features=1, width=aeParam.nFeaturesEnc, out_features=chParam.M)
    decoder = ae.Decoder_Aref(in_features=2, width=aeParam.nFeaturesDec, out_features=chParam.M)
    CEloss = nn.CrossEntropyLoss()

    # Optimizer
    optimizer = optim.Adam(list(encoder.parameters()) + list(decoder.parameters()), lr=trainingParam.learningRate)

    # AWGN Capacity
    AWGN_Cap = np.log2(1 + hlp.dB2lin(SNR_db, 'dB'))

    # Training loop
    for j in range(trainingParam.iterations):
        for i in range (trainingParam.nBatches):
            # first generate the distribution
            l_M = encoder(enc_inp)
            P_M = F.softmax(l_M, dim=1)

            # Sample indexes
            indices = sampler_t(torch.squeeze(P_M), trainingParam.batchSize) # labels
            # get onehot from sampled indices
            onehot = F.one_hot(indices.type(torch.LongTensor), 64)


            # normalization & Modulation
            constellation = tx.qammod(chParam.M)
            constellation_t = torch.tensor(constellation, dtype=torch.cfloat)
            norm_factor = torch.rsqrt(p_norm(P_M, constellation_t))
            norm_constellation = torch.mul(constellation_t, r2c(norm_factor))
            x = torch.matmul(r2c(onehot), torch.transpose(input=norm_constellation, dim0=0, dim1=1))
            should_always_be_one = p_norm(P_M, norm_constellation)

            # Channel
            noise_cpx = torch.complex(torch.randn(x.shape), torch.randn(x.shape))
            sigma2 = torch.tensor(1) / hlp.dB2lin(SNR_db, 'dB')  # 1 corresponds to the Power
            noise_snr = r2c(torch.sqrt(sigma2)) * torch.rsqrt(torch.tensor(2)) * noise_cpx
            # https://stats.stackexchange.com/questions/187491/why-standard-normal-samples-multiplied-by-sd-are-samples-from-a-normal-dist-with

            y = torch.add(x, noise_snr)

            # demodulator
            y_vec = hlp.complex2real(torch.squeeze(y))
            dec = decoder(y_vec)


            # loss
            zhat = (y_vec - hlp.complex2real(torch.squeeze(x)))
            loss = CEloss(dec, onehot.type(torch.float))
            loss_hat = loss + loss_correction_factor(F.softmax(dec, 1), zhat, sigma2)

            optimizer.zero_grad()
            loss_hat.backward()
            optimizer.step()

            MI = utils.gaussianMI(x, y, norm_constellation, chParam.M, dtype=torch.double).detach().numpy()


        # Printout and visualization
        if j % int(trainingParam.displayStep) == 0:
            print(f'epoch {j}: Loss = {loss_hat.detach().numpy() / np.log(2) :.4f} - always 1: {should_always_be_one :.2} - MI: {MI :.4f} - Cap.: {AWGN_Cap:.4f}')
        if loss < 1e-3:
            break

    # Data for the plots
    p_s_t = F.softmax(encoder(enc_inp), dim=1)
    p_s = p_s_t.detach().numpy()[0]
    constellation = tx.qammod(chParam.M)
    constellation_t = torch.tensor(constellation, dtype=torch.cfloat)
    norm_factor = torch.rsqrt(p_norm(p_s_t, constellation_t))
    norm_constellation = r2c(norm_factor) * constellation_t
    #print(p_s)
    print('Power should always be one:', p_norm(p_s_t, norm_constellation))
    plot_2D_PDF(constellation, p_s, SNR_db)

---SNR = 0.5 dB---
epoch 0: Loss = 4.9831 - always 1: 1.0 - MI: 1.0600 - Cap.: 1.0854
epoch 5: Loss = 4.8386 - always 1: 1.0 - MI: 1.0690 - Cap.: 1.0854
epoch 10: Loss = 3.7978 - always 1: 1.0 - MI: 1.0822 - Cap.: 1.0854
epoch 15: Loss = 2.9095 - always 1: 1.0 - MI: 1.0811 - Cap.: 1.0854
epoch 20: Loss = 2.3534 - always 1: 1.0 - MI: 1.0666 - Cap.: 1.0854


RuntimeError: shape '[64]' is invalid for input of size 63